### La idea es que empecemos en este File a procesar la data

In [43]:
using CSV
using DataFrames
using StatsBase

In [44]:
using DataFramesMeta
using Plots
using ImageView
using FileIO

In [45]:
using Flux

In [46]:
df = DataFrame(CSV.File("./train_labels.csv"))

Row,id,label
,String,Int64
1,f38a6374c348f90b587e046aac6079959adf3835,0
2,c18f2d887b7ae4f6742ee445113fa1aef383ed77,1
3,755db6279dae599ebb4d39a9123cce439965282d,0
4,bc3f0c64fb968ff4a8bd33af6971ecae77c75e08,0
5,068aba587a4950175d04c680d38943fd488d6a9d,0
6,acfe80838488fae3c89bd21ade75be5c34e66be7,0
7,a24ce148f6ffa7ef8eefb4efb12ebffe8dd700da,1
8,7f6ccae485af121e0b6ee733022e226ee6b0c65f,1
9,559e55a64c9ba828f700e948f6886f4cea919261,0


In [47]:
value_counts(df, col) = combine(groupby(df, col), nrow)

value_counts (generic function with 1 method)

In [48]:
value_counts(df, "label")

Row,label,nrow
,Int64,Int64
1,0,130908
2,1,89117


In [49]:
#Nos quedamos con 1000 filas para que el procesamiento sea mas rápido
df = df[sample(axes(df, 1), 1000; replace = false, ordered = true), :]

Row,id,label
,String,Int64
1,24b2e9582b5681a6e2c91fc74316f25c6ae38b81,0
2,7b5eed38e2d6de7393cbae90ba39f60f3171efe6,0
3,e5904a30c37ec6565608f38c3e9340d3c63470c1,1
4,80625e383dccdb15006d1f4b5a340b725eba7e09,1
5,fb86a744eb1cd4a343a154e4ccb0013f2a0b8474,1
6,2c8e186d0f4dd417f9737d4670d8ef1cc644d567,1
7,9b9e0ac7af2b130903137448d083247b2b1c455e,0
8,664023ea4106a18c3b418843345f24f106c37ba2,0
9,e48f8622dda1de9207ee649d214ff9661d58aaa6,0


In [50]:
es_positivo(valor::Integer) = valor == 1
positivos = filter(:label => es_positivo, df)
positivos_sample = positivos[sample(axes(positivos, 1), 5; replace = false, ordered = true), :]

Row,id,label
,String,Int64
1,5d310bcc79d619cc1b91f143630ace97a6d09451,1
2,fae7a32085ce132fcf42feda60ddcda6ea216037,1
3,94326868f5803d3ad8b200b9391f36948cc71f7f,1
4,57ac5876915fc8e6a4b5e9a35eddc45b203c665b,1
5,acdf0729ba13e35e1c4b4a31dc2f3fe8d07d5b62,1


In [51]:
es_negativo(valor::Integer) = valor == 0
negativos = filter(:label => es_negativo, df)
negativos_sample = negativos[sample(axes(negativos, 1), 5; replace = false, ordered = true), :]

Row,id,label
,String,Int64
1,d2db1bb1354f9505aca19bd45e06bf2699fdbeaa,0
2,edca1dd756f1fbb49a6601ac0c247b894a6356bf,0
3,5e38f2841fd1383cad808a37e8b30e5c649f4781,0
4,cc9e20c72cf2a2c3e83d318b992650a4396fd01e,0
5,b3cc7671d7c40a48af65bbab412d9f505aaeb16a,0


In [52]:
using Flux, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
using Printf, BSON
using ColorVectorSpace

In [53]:
df.image_path = string.("../input/histopathologic-cancer-detection/train/", df.id, ".tif")

1000-element Vector{String}:
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "1a6e2c91fc74316f25c6ae38b81.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "e7393cbae90ba39f60f3171efe6.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "6565608f38c3e9340d3c63470c1.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "b15006d1f4b5a340b725eba7e09.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "4a343a154e4ccb0013f2a0b8474.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "417f9737d4670d8ef1cc644d567.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "30903137448d083247b2b1c455e.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "18c3b418843345f24f106c37ba2.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "e9207ee649d214ff9661d58aaa6.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "ba016256dd2e0a0e0395a11b105.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "f4007740b7814574851697fd51d.tif"
 "../input/histopathologic-cancer" ⋯ 30 bytes ⋯ "b56aa329fdface9ff7bc

In [54]:
train_images = Matrix{RGB{}}[]
for row in eachrow(df)
    push!(train_images, load(row.image_path))
end

In [55]:
train_images

1000-element Vector{Matrix{RGB}}:
 [RGB{N0f8}(0.929,0.741,0.796) RGB{N0f8}(0.875,0.686,0.749) … RGB{N0f8}(1.0,0.976,0.992) RGB{N0f8}(0.796,0.714,0.741); RGB{N0f8}(0.859,0.659,0.718) RGB{N0f8}(1.0,0.898,0.965) … RGB{N0f8}(0.984,0.937,0.945) RGB{N0f8}(0.925,0.867,0.886); … ; RGB{N0f8}(0.608,0.451,0.553) RGB{N0f8}(0.741,0.573,0.678) … RGB{N0f8}(0.937,0.933,0.918) RGB{N0f8}(0.988,0.973,0.961); RGB{N0f8}(0.51,0.325,0.451) RGB{N0f8}(0.416,0.224,0.349) … RGB{N0f8}(0.976,0.988,0.961) RGB{N0f8}(0.953,0.965,0.937)]
 [RGB{N0f8}(0.847,0.651,0.859) RGB{N0f8}(0.929,0.729,1.0) … RGB{N0f8}(0.498,0.184,0.62) RGB{N0f8}(0.541,0.216,0.624); RGB{N0f8}(0.91,0.714,0.945) RGB{N0f8}(0.408,0.204,0.494) … RGB{N0f8}(0.776,0.506,0.855) RGB{N0f8}(0.835,0.553,0.875); … ; RGB{N0f8}(0.725,0.341,0.635) RGB{N0f8}(0.655,0.255,0.553) … RGB{N0f8}(0.902,0.749,0.957) RGB{N0f8}(0.773,0.659,0.871); RGB{N0f8}(0.757,0.365,0.647) RGB{N0f8}(0.753,0.337,0.627) … RGB{N0f8}(0.286,0.137,0.361) RGB{N0f8}(0.408,0.294,0.514)]
 [RGB{N0f8}

In [56]:
function make_minibatch(X, Y, idxs)
    X_batch = Array{RGB}(undef, size(X[1])..., 1, length(idxs))
    for i in 1:length(idxs)
        X_batch[:, :, :, i] = RGB.(X[idxs[i]])
    end
    Y_batch = onehotbatch(Y[idxs], 0:1)
    return (X_batch, Y_batch)
end

make_minibatch (generic function with 1 method)

In [57]:
batchsize = 128
mb_idxs = partition(1:length(train_images), batchsize)
train_set = [make_minibatch(train_images, df.label, i) for i in mb_idxs]

8-element Vector{Tuple{Array{RGB, 4}, OneHotArrays.OneHotMatrix{UInt32, 2, Vector{UInt32}}}}:
 ([RGB{N0f8}(0.929,0.741,0.796) RGB{N0f8}(0.875,0.686,0.749) … RGB{N0f8}(1.0,0.976,0.992) RGB{N0f8}(0.796,0.714,0.741); RGB{N0f8}(0.859,0.659,0.718) RGB{N0f8}(1.0,0.898,0.965) … RGB{N0f8}(0.984,0.937,0.945) RGB{N0f8}(0.925,0.867,0.886); … ; RGB{N0f8}(0.608,0.451,0.553) RGB{N0f8}(0.741,0.573,0.678) … RGB{N0f8}(0.937,0.933,0.918) RGB{N0f8}(0.988,0.973,0.961); RGB{N0f8}(0.51,0.325,0.451) RGB{N0f8}(0.416,0.224,0.349) … RGB{N0f8}(0.976,0.988,0.961) RGB{N0f8}(0.953,0.965,0.937);;;; RGB{N0f8}(0.847,0.651,0.859) RGB{N0f8}(0.929,0.729,1.0) … RGB{N0f8}(0.498,0.184,0.62) RGB{N0f8}(0.541,0.216,0.624); RGB{N0f8}(0.91,0.714,0.945) RGB{N0f8}(0.408,0.204,0.494) … RGB{N0f8}(0.776,0.506,0.855) RGB{N0f8}(0.835,0.553,0.875); … ; RGB{N0f8}(0.725,0.341,0.635) RGB{N0f8}(0.655,0.255,0.553) … RGB{N0f8}(0.902,0.749,0.957) RGB{N0f8}(0.773,0.659,0.871); RGB{N0f8}(0.757,0.365,0.647) RGB{N0f8}(0.753,0.337,0.627) … RGB{N0f8

In [58]:
model = Chain(
        Conv((3, 3), 1=>32, pad=(1,1), relu),
        MaxPool((3,3)),
        Conv((3, 3), 32=>64, pad=(1,1), relu),
        MaxPool((3,3)),
        Conv((3, 3), 64=>128, pad=(1,1), relu),
        MaxPool((3,3)),
        flatten,
        Dense(15488, 2),
        softmax)

LoadError: MethodError: objects of type Module are not callable